In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target
X_feature_names = ['age', 'gender', 'body mass index', 'average blood pressure','bl_0','bl_1','bl_2','bl_3','bl_4','bl_5']
#bin target variable for better sampling
bins = 50*np.arange(8)
binned_y = np.digitize(y, bins)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=binned_y)

In [3]:
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV

In [4]:
 param_dist = {
 'max_samples': [0.5,1.0],
 'max_features' : [0.5,1.0],
 'oob_score' : [True, False],
 'base_estimator__n_neighbors': [3,5],
 'n_estimators': [100]
 }

In [5]:
single_estimator = KNeighborsRegressor()
ensemble_estimator = BaggingRegressor(base_estimator = single_estimator)

In [6]:
pre_gs_inst_bag = RandomizedSearchCV(ensemble_estimator,
    param_distributions = param_dist,
    cv=3,
    n_iter = 5,
    n_jobs=-1)
pre_gs_inst_bag.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=BaggingRegressor(base_estimator=KNeighborsRegressor()),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'base_estimator__n_neighbors': [3, 5],
                                        'max_features': [0.5, 1.0],
                                        'max_samples': [0.5, 1.0],
                                        'n_estimators': [100],
                                        'oob_score': [True, False]})

In [7]:
pre_gs_inst_bag.best_params_

{'oob_score': True,
 'n_estimators': 100,
 'max_samples': 1.0,
 'max_features': 0.5,
 'base_estimator__n_neighbors': 3}

In [8]:
rs_bag = BaggingRegressor(**{'max_features': 1.0,
    'max_samples': 0.5,
    'n_estimators': 1000,
    'oob_score': True,
    'base_estimator': KNeighborsRegressor(n_neighbors=5)})
rs_bag.fit(X_train, y_train)

BaggingRegressor(base_estimator=KNeighborsRegressor(), max_samples=0.5,
                 n_estimators=1000, oob_score=True)

In [9]:
y_pred = rs_bag.predict(X_test)
from sklearn.metrics import r2_score, mean_absolute_error
print ("R-squared",r2_score(y_test, y_pred))
print ("MAE : ",mean_absolute_error(y_test, y_pred))
print ("MAPE : ",(np.abs(y_test - y_pred)/y_test).mean())

R-squared 0.44859494916718257
MAE :  45.777950561797745
MAPE :  0.45537247650359397
